In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps


# API Keys
from config import gkey



In [2]:
active_stores_df = pd.read_csv("liquor_store_ia.csv")
active_stores_df.head()

,Unnamed: 0,Address,City,Status,State,Zip Code,lat,lng
0,0,200 State Po Box 98,Guthrie Center,A,IA,50115.0,41.677408,-94.502649
1,1,1111 8TH ST,Boone,A,IA,50036.0,42.064750,-93.876110
2,2,3140 Agency,Burlington,A,IA,52601.0,40.816170,-91.136651
3,3,939 Angular,Burlington,A,IA,52601.0,40.801319,-91.110647
4,4,"1843 Johnson Avenue, N.W.",Cedar Rapids,A,IA,52405.0,41.973912,-91.698396


In [3]:
locations = active_stores_df[["lat", "lng"]].astype(float)
# Create sotre symbol layer
store_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,)
    #info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]

fig = gmaps.figure()
fig.add_layer(store_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
counties_df = pd.read_csv("2017_Poverty_Data.csv")

In [5]:
counties_df['lat'] = ""
counties_df['lng'] = ""

In [6]:
params = {"key":gkey}
for index, row in counties_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    county_type = row['County']
    state_type = row['State']
    params['address'] = f"{county_type},{state_type}"
    county_lat_lng = requests.get(base_url, params=params).json()
    counties_df.loc[index, "lat"] = county_lat_lng["results"][0]["geometry"]["location"]["lat"]
    counties_df.loc[index, "lng"] = county_lat_lng["results"][0]["geometry"]["location"]["lng"]

In [7]:
gmaps.configure(api_key=gkey)

locations = counties_df[["lat", "lng"]].astype(float)

poverty_rate = counties_df["Total Poverty Rate (%)"].astype(float)

In [8]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
heat_layer.dissipating = False
heat_layer.max_intensity = 90
heat_layer.point_radius = 1

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(store_layer)

fig

Figure(layout=FigureLayout(height='420px'))

AttributeError: 'Figure' object has no attribute 'savefig'